In [15]:
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

Train Data Prep

In [31]:
def data_prep(filename):
    df = pd.read_parquet(filename)
    print(df.shape[1])
    count_all = df.shape[0]
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    print(np.std(df['duration']))
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    count_filt = df.shape[0]
    print(count_filt/count_all)
    return df


In [32]:
train_filename = "data/yellow_tripdata_2022-01.parquet"
val_filename = "data/yellow_tripdata_2022-02.parquet"
target = 'duration'

dv = DictVectorizer()

train_df = data_prep(train_filename)
categorical = ['PULocationID', 'DOLocationID']
train_df[categorical] = train_df[categorical].astype(str)
train_dict = train_df[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
print(X_train.shape)
y_train = train_df[target].values

val_df = data_prep(val_filename)
val_df[categorical] = val_df[categorical].astype(str)
val_dict = val_df[categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_val = val_df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
rmse

19
46.44529571272228
0.9827547930522406
(2421440, 515)
19
47.26393283611246
0.979444397269143


6.986191065263967

In [29]:
y_pred = lr.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

7.786413838484518